[![pythonista](img/pythonista.png)](https://www.pythonista.io)

# *SQLAlchemy* asíncrono.

*SQLAlchemy* cuenta con una extensión compatible con ```asyncio``` mediante una extensión especializada.

https://docs.sqlalchemy.org/en/latest/orm/extensions/asyncio.html

In [ ]:
!pip install sqlalchemy[asyncio] aiosqlite

In [ ]:
from sqlalchemy.ext.asyncio import create_async_engine

In [ ]:
engine = create_async_engine('sqlite:///db.sqlite3')

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>